In [125]:
# pip install fsspec / conda install -c conda-forge fsspec
# pip install s3fs
import pandas as pd
import numpy as np
import boto3
import os
from sagemaker import get_execution_role

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment'
file_name = 'payment.csv'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [126]:
df = pd.read_csv(data_location)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head(3)

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,1,39,111.111.000.039,TERM-245,225.0,227.0,CREDIT,VIRTUAL,2024-01-01 09:01:21.000 -0300,MOTOR,2340.0,day,0,wkday,0,1,19,555.26,19,555.26,19,555.26,157,0,1
1,2,39,111.111.000.039,TERM-235,293.0,238.0,CREDIT,VIRTUAL,2024-01-01 09:03:58.000 -0300,GIFT,423.0,day,0,wkday,0,1,19,555.26,19,555.26,19,555.26,72,157,1
2,3,135,111.111.000.135,TERM-25,65.0,81.0,CREDIT,CHIP,2024-01-01 09:05:10.000 -0300,SPORTING,389.0,day,0,wkday,0,1,10,227.60,10,227.60,10,227.60,4,0,0


In [127]:
df = df.filter(['fraud',
                'payment_at',
                'terminal_name',
                'coord_x',
                'coord_y',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [128]:
df.head(3)

,fraud,payment_at,terminal_name,coord_x,coord_y,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,2024-01-01 09:01:21.000 -0300,TERM-245,225.0,227.0,CREDIT,VIRTUAL,MOTOR,2340.0,19,555.26,19,555.26,19,555.26,0
1,1,2024-01-01 09:03:58.000 -0300,TERM-235,293.0,238.0,CREDIT,VIRTUAL,GIFT,423.0,19,555.26,19,555.26,19,555.26,157
2,0,2024-01-01 09:05:10.000 -0300,TERM-25,65.0,81.0,CREDIT,CHIP,SPORTING,389.0,10,227.60,10,227.60,10,227.60,0


In [129]:
# featuring enginering

# Create a new column distance
import math

df.insert(5, "distance", 0)
for ind in df.index:
    p1 = [0, 0]
    p2 = [df['coord_x'][ind], df['coord_y'][ind]]
    distance = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
    df["distance"][ind] = distance

/tmp/ipykernel_1106/2151328808.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["distance"][ind] = distance
/tmp/ipykernel_1106/2151328808.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '319.61539387207245' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df["distance"][ind] = distance


In [130]:
# extract hour
df["ts_payment_at"] = pd.to_datetime(df['payment_at'])
print(df.ts_payment_at.dtype)
df["hour"] = df.ts_payment_at.dt.hour

datetime64[ns, UTC-03:00]


In [131]:
df = df.filter(['fraud',
                'terminal_name',
                'hour',
                'distance',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [132]:
df.head(3)

,fraud,terminal_name,hour,distance,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,TERM-245,9,319.615394,CREDIT,VIRTUAL,MOTOR,2340.0,19,555.26,19,555.26,19,555.26,0
1,1,TERM-235,9,377.482450,CREDIT,VIRTUAL,GIFT,423.0,19,555.26,19,555.26,19,555.26,157
2,0,TERM-25,9,103.855669,CREDIT,CHIP,SPORTING,389.0,10,227.60,10,227.60,10,227.60,0


In [133]:
df = pd.get_dummies(df, columns=['card_model', ], prefix='card_model', dtype=int, drop_first=False)
df = pd.get_dummies(df, columns=['card_type', ], prefix='card_type', dtype=int, drop_first=False)
df.head(3)

,fraud,terminal_name,hour,distance,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT
0,1,TERM-245,9,319.615394,MOTOR,2340.0,19,555.26,19,555.26,19,555.26,0,0,1,1
1,1,TERM-235,9,377.482450,GIFT,423.0,19,555.26,19,555.26,19,555.26,157,0,1,1
2,0,TERM-25,9,103.855669,SPORTING,389.0,10,227.60,10,227.60,10,227.60,0,1,0,1


In [134]:
df = df.filter(['fraud',
                'distance',
                'card_model_CHIP',
                'card_model_VIRTUAL',
                'card_type_CREDIT',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [135]:
df.head(3)

,fraud,distance,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,319.615394,0,1,1,2340.0,19,555.26,19,555.26,19,555.26,0
1,1,377.482450,0,1,1,423.0,19,555.26,19,555.26,19,555.26,157
2,0,103.855669,1,0,1,389.0,10,227.60,10,227.60,10,227.60,0
